In [ ]:
import numpy as np

In [ ]:
M = 64 # symbols per frame
N = 8 # modulation order
number_of_frames = 100
modulation_order = 4 # bits per symbol

In [ ]:
snr_db = list(range(-20, 21, 5))
print(snr_db)

In [ ]:
binary_sequence = np.random.randint(0, 2, size=64*8)
print(binary_sequence)

In [ ]:
# Reshape into (M, N)
binary_matrix = binary_sequence.reshape(M, N)
print(binary_matrix)

In [ ]:
# Convert each row to a decimal number
decimal_vector = np.array([int(''.join(map(str, row)), 2) for row in binary_matrix])
# # Reshape to (64, 1)
# decimal_vector = decimal_vector.reshape(-1, 1)
print(decimal_vector)

In [ ]:
import numpy as np

def qam4_modulate(bits):
    """
    4-QAM modulation: Maps 2-bit pairs to QAM symbols.

    Parameters:
    bits (numpy array): 1D binary array (length should be even)

    Returns:
    numpy array: Complex symbols representing QAM modulation
    """
    if len(bits) % 2 != 0:
        raise ValueError("Input length must be even for 4-QAM modulation.")

    bit_pairs = bits.reshape(-1, 2)  # Reshape into pairs
    symbol_map = {
        (0, 0):  1+1j,  # 00 -> (1,1)
        (0, 1):  1-1j,  # 01 -> (1,-1)
        (1, 0): -1+1j,  # 10 -> (-1,1)
        (1, 1): -1-1j   # 11 -> (-1,-1)
    }
    
    symbols = np.array([symbol_map[tuple(pair)] for pair in bit_pairs])
    return symbols

def qam4_demodulate(symbols):
    """
    4-QAM demodulation: Maps QAM symbols back to binary sequence.

    Parameters:
    symbols (numpy array): Complex-valued matrix received from QAM modulation

    Returns:
    numpy array: 2D binary array (same shape as input but with each element mapped to 2 bits)
    """
    decision_map = {
        1+1j: [0, 0],  # 00
        1-1j: [0, 1],  # 01
        -1+1j: [1, 0],  # 10
        -1-1j: [1, 1]   # 11
    }
    
    # Vectorized processing of each symbol in the input matrix
    demodulated_bits = np.zeros((symbols.shape[0], symbols.shape[1] * 2), dtype=int)
    
    for i in range(symbols.shape[0]):
        for j in range(symbols.shape[1]):
            symbol = symbols[i, j]
            closest_point = min(decision_map.keys(), key=lambda x: abs(x - symbol))
            demodulated_bits[i, 2*j:2*j+2] = decision_map[closest_point]
    
    return demodulated_bits
# Example Usage
np.random.seed(42)  # For reproducibility
input_bits = np.random.randint(0, 2, size=16)  # Random binary sequence of length 16
modulated_symbols = qam4_modulate(input_bits)  # QAM Modulation

In [ ]:
qam4_symbols = qam4_modulate(bits=binary_matrix)
print(qam4_symbols)

In [ ]:
qam4_symbols_reshaped = qam4_symbols.reshape(64, 4)
print(qam4_symbols_reshaped)

In [ ]:
I = np.eye(64)
print(I)

In [ ]:
def dft_matrix(N):
    """
    Generates an N x N Discrete Fourier Transform (DFT) matrix.

    Parameters:
    N (int): Size of the DFT matrix.

    Returns:
    numpy.ndarray: N x N DFT matrix.
    """
    n = np.arange(N)
    k = n.reshape((N, 1))
    omega = np.exp(-2j * np.pi * k * n / N)  # DFT formula
    return omega

# Generate a 4x4 DFT matrix
dft_4x4 = dft_matrix(4)
print(dft_4x4)

In [ ]:
S = np.kron(I, dft_4x4)
print(S)
print(len(S))

In [ ]:
S_vector = np.reshape(256 * 256, 1)
print(S_vector)

In [ ]:
# Channel generation
tau = [0, 1, 2, 3]
mu = [0, 1, 2, 3]

In [ ]:

def generate_complex_gaussian_matrix(rows, cols, sigma):
    """
    Generates a matrix of complex Gaussian random numbers.

    Parameters:
    - rows (int): Number of rows.
    - cols (int): Number of columns.
    - sigma (float): Standard deviation (square root of variance).

    Returns:
    - numpy.ndarray: (rows, cols) matrix of complex Gaussian numbers.
    """
    real_part = np.random.normal(0, sigma / np.sqrt(2), (rows, cols))  # Real part ~ N(0, sigma^2/2)
    imag_part = np.random.normal(0, sigma / np.sqrt(2), (rows, cols))  # Imaginary part ~ N(0, sigma^2/2)

    complex_matrix = real_part + 1j * imag_part
    return complex_matrix

In [ ]:
sigma = 1
complex_gaussian_matrix = generate_complex_gaussian_matrix(64, 4, sigma)
print(complex_gaussian_matrix)

In [ ]:
from scipy.linalg import dft

def generate_otfs_receiver_noise(N, M, sigma):
    """
    Generates receiver-processed noise for an OTFS system.

    Parameters:
    N (int): Size of the identity and DFT matrices (e.g., 64)
    M (int): Number of columns in the noise matrix
    sigma (float): Standard deviation of the Gaussian noise

    Returns:
    numpy.ndarray: Processed noise matrix (N x M)
    """
    # Step 1: Generate Identity Matrix (N x N)
    I_N = np.eye(N)

    # Step 2: Generate DFT Matrix (N x N)
    F_N = dft(N)  # DFT matrix

    # Step 3: Generate Complex Gaussian Noise Matrix (N x M)
    W = np.random.normal(0, sigma, (N, M)) + 1j * np.random.normal(0, sigma, (N, M))

    # Step 4: Compute Processed Noise
    N_processed = np.conj(F_N.T) @ W @ I_N  # Apply transformations

    return N_processed

In [ ]:
# Noise generation

In [ ]:
pi = np.roll(np.eye(64 * 4), shift=1)
print(pi)

In [ ]:
# Define the diagonal elements
i_values = np.arange(64 * 4)  # i ranges from 0 to 255
diagonal_elements = np.exp(1j * i_values * 2 * np.pi / 64)  # e^(i*j*2*pi/64)

# Create a diagonal matrix using np.diag()
delta = np.diag(diagonal_elements)

# Print the result
print("Diagonal Matrix:\n", delta)

In [ ]:
H = np.matmul(np.roll(pi, shift=0), np.linalg.matrix_power(delta, 0))
for i in range(1, 4):
    H = np.add(np.matmul(np.roll(pi, i), np.linalg.matrix_power(delta, i)), H)

print(H)

In [ ]:
H_eff = np.matmul(np.matmul(np.kron(dft_4x4, I), H), np.kron(np.conj(dft_4x4), I))
print(H_eff)

In [ ]:
print(len(H_eff))
print(len(H_eff[0]))

In [ ]:
print(len(qam4_symbols))

In [ ]:
complex_gaussian_matrix_reshaped = complex_gaussian_matrix.reshape(64 * 4, 1)
print(complex_gaussian_matrix_reshaped)

In [ ]:
w_tilde = np.matmul(np.kron(dft_4x4, I), complex_gaussian_matrix_reshaped)
print(w_tilde)
print(len(w_tilde))
print(len(w_tilde[0]))

In [ ]:
qam4_symbols_256_1 = qam4_symbols.reshape(256, 1)
print(len(qam4_symbols_256_1))
print(len(qam4_symbols_256_1[0]))
y = (H_eff @ qam4_symbols_256_1) + w_tilde
print(y)
print(len(y))
print(len(y[0]))

In [ ]:
x_cap = np.matmul(np.linalg.inv(H_eff), y)
print(x_cap)

In [ ]:
x_cap_demodulated = qam4_demodulate(x_cap)
print(x_cap_demodulated)

In [ ]:
print(len(x_cap_demodulated))
print(len(x_cap_demodulated[0]))

In [ ]:
x_cap_demodulated_512 = x_cap_demodulated.reshape(256 * 2, 1)
print(x_cap_demodulated_512)
print(len(x_cap_demodulated_512))
print(len(x_cap_demodulated_512[0]))

In [ ]:
count:int = 0
for i in range(len(x_cap_demodulated_512)):
    if(x_cap_demodulated_512[i] != binary_sequence[i]):
        count += 1
print(count)